In [1]:
import settings
from datetime import date, datetime
from sentinelsat_download import SentinelsatProducts
import time

In [2]:
footprint = settings.footprint
data_path = settings.data_path

### Download all data between two dates

In [ ]:
from IPython.display import clear_output

date1 = date(2021, 1, 1)
date2 = date(2021, 12, 31)

while True:
    try:
        datos = SentinelsatProducts(date1, date2, footprint=footprint, 
                                    platformname="Sentinel-3", instrument="SLSTR", 
                                    level="L2", p_type="SL_2_LST___")
        #datos.filter_products(instrument="SLSTR", level="L2", p_type="SL_2_LST___", timeliness="Non Time Critical")
        datos.download_products(data_path)
    except Exception as e:
        clear_output(wait=True)
        print("/"*20)
        print(str(e))
        print("/"*20)
        time.sleep(60*10)

### Download data from valid Sentinel-2 days

In [ ]:
from IPython.display import clear_output
from process_sentinel2 import Mask, DayDataGenerator, DayData
import traceback

DATA_PATH = settings.data_path
DATE_FORMAT = '%Y-%m-%d'
START_DATE = '2016-12-21'
END_DATE = '2022-04-20'

data_generator = DayDataGenerator(START_DATE, END_DATE, DATE_FORMAT, DATA_PATH, skip_invalid=True)
valid_dates = [day.date.date() for day in data_generator]
print("Total days to download:", len(valid_dates))
while True:
    date1 = date(2016, 12, 21)
    date2 = date(2022, 4, 20)
    try:
        datos = SentinelsatProducts(date1, date2, footprint=footprint, 
                                    platformname="Sentinel-3", instrument="SLSTR", 
                                    level="L2", p_type="SL_2_LST___")
        datos.filter_by_dates(valid_dates)
        datos.download_products(data_path)
    except Exception as e:
        clear_output(wait=True)
        print("/"*20)
        print(traceback.format_exc())
        print("/"*20)
        time.sleep(60*10)

In [ ]:
import os

def get_folder_size(folder_path):
    size = 0
    for path, dirs, files in os.walk(folder_path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.path.getsize(fp)/1024/1024
    return size

def get_sentinel_folder_datetime(folder_path):
    folder_name = os.path.basename(folder_path)
    return datetime.strptime(folder_name.split("____")[1].split("_")[0][0:15], "%Y%m%dT%H%M%S")
    

dirs_to_delete = {}
for day_dir in os.listdir(DATA_PATH):
    sen3_dirs = [os.path.join(DATA_PATH, day_dir, sat_data) for sat_data in os.listdir(os.path.join(DATA_PATH, day_dir)) if sat_data.endswith("SEN3")]
    sen3_dir_size = [get_folder_size(s3_data) for s3_data in sen3_dirs]    
    sen3_dir_dt = [get_sentinel_folder_datetime(s3_data) for s3_data in sen3_dirs]    
    
    small_dirs = [[sen3_dirs[i], dir_size] for i, dir_size in enumerate(sen3_dir_size) if dir_size < 1000]
    big_dirs = [[sen3_dirs[i], dir_size] for i, dir_size in enumerate(sen3_dir_size) if dir_size > 1000]
    
    if len(small_dirs) > 0:
        for big_dir in big_dirs:
            dirs_to_delete[big_dir[0]] = big_dir[1]  